# 데이터 정제 및 준비

In [2]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 누락 데이터 처리하기

In [3]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### 누락 데이터 골라내기 : dropna()

In [5]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [6]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [8]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()

In [9]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [10]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [11]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [12]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [13]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [14]:
df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [15]:
df.dropna()

,0,1,2
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [16]:
# 몇 개의 이상값이 들어있는 로우만 볼 경우
df.dropna(thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


### 결측치 채우기: fillna()

In [ ]:
df.fillna(0)

In [ ]:
df.fillna({1: 0.5, 2: 0})

In [ ]:
df.fillna(df.mean())

## 데이터 변형

### 중복 제거: drop_duplicates()

In [17]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [18]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [19]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [20]:
data['v1'] = range(7)
#k1 컬럼에 기반해서 중복 제거
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [ ]:
# 중복값 중, 마지막으로 발견된 값을 유지
data.drop_duplicates(['k1', 'k2'], keep='last')

### 함수나 매핑을 이용해서 데이터 변형하기 : map()

In [21]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [22]:
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [23]:
data['animal'] = data['food'].str.lower().map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [24]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 값 치환하기 : replace()

In [25]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [26]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [27]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [28]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [29]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 컬럼/색인 이름 바꾸기: rename

In [30]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [31]:
transform = lambda x: x[:4].upper()
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [32]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [33]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [34]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [35]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 개별화(Discretization) and 양자화(Binning)

In [36]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [37]:
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [38]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [39]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [40]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [41]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [42]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

### 특잇값 찾고 제외하기

In [43]:
data = pd.DataFrame(np.random.randn(1000, 4))
data

,0,1,2,3
0,0.476985,3.248944,-1.021228,-0.577087
1,0.124121,0.302614,0.523772,0.000940
2,1.343810,-0.713544,-0.831154,-2.370232
3,-1.860761,-0.860757,0.560145,-1.265934
4,0.119827,-1.063512,0.332883,-2.359419
...,...,...,...,...
995,-0.997174,0.046486,-0.610441,-0.394982
996,1.199915,-0.451814,-0.155385,-0.153514
997,0.011194,-0.050555,0.420211,1.190981
998,1.561488,-1.132925,0.097083,-1.031573


In [44]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.066679,0.020188,-0.000651,-0.067915
std,0.992674,1.004784,0.995859,0.995834
min,-3.548824,-3.184377,-3.745356,-3.428254
25%,-0.596286,-0.648915,-0.642609,-0.774890
50%,0.094503,-0.001593,-0.012007,-0.117489
75%,0.780282,0.674685,0.654328,0.616366
max,2.653656,3.260383,3.927528,3.366626


In [45]:
col = data[2]
col[np.abs(col) > 3]

92     3.927528
300   -3.399312
395   -3.745356
Name: 2, dtype: float64

In [46]:
# 절대값이 3을 초과하는 값이 들어있는 모든 로우 선택
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
0,0.476985,3.248944,-1.021228,-0.577087
92,0.552936,0.106061,3.927528,-0.255126
97,-0.565230,3.176873,0.959533,-0.975340
300,0.457246,-0.025907,-3.399312,-0.974657
319,1.951312,3.260383,0.963301,1.201206
395,0.508391,-0.196713,-3.745356,-1.520113
494,-0.242459,-3.056990,1.918403,-0.578828
517,0.682841,0.326045,0.425384,-3.428254
581,1.179227,-3.184377,1.369891,-1.074833
803,-3.548824,1.553205,-2.186301,1.277104


In [47]:
# 절대값이 3을 초과하는 수는 3 또는 -3으로 설정
# sign(data) : 양수(1) or 음수(-1)인지 판단
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.067228,0.019744,-0.000434,-0.067853
std,0.990823,1.001918,0.988794,0.993278
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.596286,-0.648915,-0.642609,-0.774890
50%,0.094503,-0.001593,-0.012007,-0.117489
75%,0.780282,0.674685,0.654328,0.616366
max,2.653656,3.000000,3.000000,3.000000


## 연습문제

- 자유롭게 데이터전처리하기

    [예시]
    - 컬럼명 변경하기
    - 날짜 데이터 파싱하기
    - 이상치 처리하기
    - 계절, 날씨를 text로 변경하기(예: 봄, 여름, 가을, 겨울)
    - 자전거 대여횟수를 카테고리화 하기(예: 매우 적음, 적음, 보통, 많음, 매우 많음)
    - atemp 값을 소수점 2자리까지 보여지도록 바꾸기
    
    
- 데이터전처리 과정을 마친 데이터프레임을 csv 형태로 저장

- 코드제출

    https://drive.google.com/drive/folders/149Ryp8xzvMnklI4bY7kF074tSsr5-H0c?usp=sharing
    
    


In [48]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# 읽어드릴 파일 경로 : /dataset/bike_sharing_demand_train.csv
filepath = "./datasets/exercise/bike_sharing_demand_train.csv"
data = pd.read_csv(filepath)
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [50]:
data.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10886.000000,10886.000000,10886.000000,10886.000000,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000,10886.000000
mean,2.506614,0.028569,0.680875,1.418427,20.23086,23.655084,61.886460,12.799395,36.021955,155.552177,191.574132
std,1.116174,0.166599,0.466159,0.633839,7.79159,8.474601,19.245033,8.164537,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.82000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.94000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.50000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,4.000000,0.000000,1.000000,2.000000,26.24000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.00000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


In [64]:
data.sort_values('temp', ascending=False).head(10)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
8311,2012-07-07 16:00:00,3,0,0,1,41.00,43.180,19,11.0014,102,192,294
8309,2012-07-07 14:00:00,3,0,0,2,39.36,43.180,30,8.9981,105,203,308
8307,2012-07-07 12:00:00,3,0,0,1,39.36,43.180,31,23.9994,124,218,342
8308,2012-07-07 13:00:00,3,0,0,2,39.36,43.180,31,16.9979,116,244,360
8312,2012-07-07 17:00:00,3,0,0,1,39.36,42.425,26,8.9981,103,176,279
8310,2012-07-07 15:00:00,3,0,0,1,39.36,42.425,26,0.0000,113,193,306
8333,2012-07-08 14:00:00,3,0,0,1,39.36,45.455,36,11.0014,77,235,312
8306,2012-07-07 11:00:00,3,0,0,1,38.54,43.940,38,12.9980,122,245,367
8572,2012-07-18 13:00:00,3,0,1,1,38.54,42.425,33,0.0000,49,154,203
8573,2012-07-18 14:00:00,3,0,1,1,38.54,43.940,38,22.0028,54,116,170


### 결측치 제거

In [71]:
data = data.dropna()

In [81]:
True in data.duplicated()

True

### 중복값 제거

In [85]:
dup_list = data.duplicated()
dup_list[dup_list == True]

Series([], dtype: bool)

In [94]:
data = data.drop(data.loc[data.weather == 4].index)

In [95]:
data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10881,2012-12-19 19:00:00,4,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,4,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,4,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,4,0,1,1,13.94,17.425,61,6.0032,12,117,129


In [104]:
def change_season(val):
    if val==1:
        season = 'spring'
    elif val == 2:
        season = 'summer'
    elif val == 3:
        season = 'autumn'
    elif val == 4:
        season = 'winter'
    return season

data.season = data.season.apply(lambda x : change_season(x))

In [106]:
data.tail()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
10881,2012-12-19 19:00:00,winter,0,1,1,15.58,19.695,50,26.0027,7,329,336
10882,2012-12-19 20:00:00,winter,0,1,1,14.76,17.425,57,15.0013,10,231,241
10883,2012-12-19 21:00:00,winter,0,1,1,13.94,15.910,61,15.0013,4,164,168
10884,2012-12-19 22:00:00,winter,0,1,1,13.94,17.425,61,6.0032,12,117,129
10885,2012-12-19 23:00:00,winter,0,1,1,13.12,16.665,66,8.9981,4,84,88


### 자전거 대여횟수를 카테고리화 하기

In [154]:
count_list = data['count'].to_list()

In [143]:
cat_list = [0, 50, 150, 300 ,450, 1000]
cats = pd.cut(count_list, cat_list)

In [144]:
cats

[(0, 50], (0, 50], (0, 50], (0, 50], (0, 50], ..., (300, 450], (150, 300], (150, 300], (50, 150], (50, 150]]
Length: 10885
Categories (5, interval[int64]): [(0, 50] < (50, 150] < (150, 300] < (300, 450] < (450, 1000]]

In [145]:
cats.codes

array([0, 0, 0, ..., 2, 1, 1], dtype=int8)

In [146]:
cats.categories

IntervalIndex([(0, 50], (50, 150], (150, 300], (300, 450], (450, 1000]],
              closed='right',
              dtype='interval[int64]')

In [147]:
data['count_cat'] = cats.codes

In [148]:
def change_cat_name(val):
    if val == 0:
        cat_name = '매우적음'
    elif val == 1:
        cat_name = '적음'
    elif val == 2:
        cat_name = '보통'
    elif val == 3:
        cat_name = '많음'
    elif val == 4:
        cat_name = '매우많음'
    else:
        cat_name = '?'
    return cat_name

data.count_cat = data.count_cat.apply(lambda x : change_cat_name(x))


### 대여 대수 분포

In [149]:
data.count_cat.value_counts()

매우적음    2969
보통      2832
적음      2602
많음      1379
매우많음    1103
Name: count_cat, dtype: int64

In [150]:
data.describe()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,10885.000000,10885.000000,10885.000000,10885.000000,10885.000000,10885.000000,10885.000000,10885.000000,10885.000000,10885.000000
mean,0.028571,0.680845,1.418190,20.231965,23.656213,61.884244,12.800020,36.024713,155.551952,191.576665
std,0.166606,0.466171,0.633384,7.791094,8.474171,19.244529,8.164652,49.961943,151.045970,181.152582
min,0.000000,0.000000,1.000000,0.820000,0.760000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,1.000000,13.940000,16.665000,47.000000,7.001500,4.000000,36.000000,42.000000
50%,0.000000,1.000000,1.000000,20.500000,24.240000,62.000000,12.998000,17.000000,118.000000,145.000000
75%,0.000000,1.000000,2.000000,26.240000,31.060000,77.000000,16.997900,49.000000,222.000000,284.000000
max,1.000000,1.000000,3.000000,41.000000,45.455000,100.000000,56.996900,367.000000,886.000000,977.000000


### groupby를 사용하여 공휴일, 날씨 대비 대여량 비교 표 생성

In [153]:
grouped = data.groupby(['workingday', 'weather'])
grouped.mean()[['casual', 'registered', 'count']]

casual  registered       count
workingday weather                                   
0          1        65.540161  137.805780  203.345941
           2        49.307692  117.221851  166.529543
           3        33.892857   86.741071  120.633929
1          1        28.039678  178.116553  206.156231
           2        22.208054  162.501807  184.709861
           3        11.639370  106.576378  118.215748